I'm using the results obtained from a blend: 
* https://www.kaggle.com/domizianostingi/volcanic-blend

In [ ]:
!pip install ProgressBar

In [ ]:
import numpy as np 
import pandas as pd 
import glob 
from progressbar import ProgressBar
from sklearn.decomposition import PCA as pca
import os 

In [ ]:

def prepare(name,features):
    train_df = pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/train.csv')
    index=[]
    df=[]  # 4431, 10 * number of featuers
    frag = glob.glob("../input/predict-volcanic-eruptions-ingv-oe/{}/*".format(name)) # len = 4431
    
    pbar = ProgressBar()
    for i in pbar(frag[0:4]):
        for j in features:
            df = np.append(df,pd.read_csv(i).agg(j))
    
    df = pd.DataFrame(df.reshape(len(frag[0:4]),len(features)*10))  
    
    col_name=[]
    num = [0,1,2,3,4,5,6,7,8,9]
    for j in features:
        for i in num:
            col_name=np.append(col_name,'{}_{}'.format(j,i))
            
    df.columns=col_name 
    
    for i in range(0,len(frag[0:4])):
        index = np.append(index,os.path.splitext(frag[i].split('{}/'.format(name))[1])[0])
        
    df['segment_id']=index
    df['segment_id']=df['segment_id'].astype(int)
    
    if name == 'train': 
        df = pd.merge(df, train_df, on =['segment_id'],how='left') 
        
    return(df)

In [ ]:
features = (('mean'),('std'),('min'),('max'))

In [ ]:
%%time
train = prepare('train',features)

In [ ]:
%%time
test = prepare('test',features)

In [ ]:
def fill_na(data):
    for i in data.columns:
        data[i] = data[i].fillna(np.mean(train[i]))

In [ ]:
fill_na(train)
fill_na(test)

In [ ]:
pseudo = pd.read_csv('../input/volcanic-blend/submission.csv')

In [ ]:
pseudo_test = pd.merge(test,pseudo, on =['segment_id'], how='left')

In [ ]:
pseudo_train = pd.concat((train,pseudo_test),axis=0)

In [ ]:
y = pseudo_train['time_to_eruption']
x = pseudo_train.iloc[:,0:-2]

In [ ]:
from sklearn.metrics import mean_absolute_error as mae
from  sklearn.tree import DecisionTreeClassifier
from  sklearn.model_selection import train_test_split
import xgboost as xgb

model = xgb.XGBRegressor(n_estimators=500000,max_depth=8,learning_rate=0.05,alpha=0.1,SUBSAMPLE=0.6)#,tree_method='gpu_hist'
Xt, Xv, Yt, Yv = train_test_split(x, y, test_size =0.2, shuffle=False)
eval_set = [(Xv,Yv)]
model.fit(Xt, Yt,early_stopping_rounds=25,eval_metric='mae', eval_set=eval_set, verbose=False)
prediction = model.predict(test.iloc[:,0:-1])
print('mean absolute error: ', mae(prediction,pseudo['time_to_eruption']))

In [ ]:
sub_df = pd.DataFrame(test['segment_id'])
sub_df = pd.concat([sub_df,pd.Series(prediction)],axis=1)
sample_submission=pd.read_csv('../input/predict-volcanic-eruptions-ingv-oe/sample_submission.csv')
        
sample_submission = pd.merge(sample_submission,sub_df, on =['segment_id'])
sample_submission = sample_submission.drop(columns=['time_to_eruption'])
sample_submission.columns = ['segment_id', 'time_to_eruption']
sample_submission.to_csv('sample_submission.csv', header=True, index=False)
print('saved')